In [1]:
import requests
import pandas as pd
import json
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "notebook_connected"

In [2]:
#Marketing team wants to focus first on the best cities to travel to in France. According One Week In.com here are the top-35 cities to visit in France:
listofcities=["Mont Saint Michel",
"St Malo",
"Bayeux",
"Le Havre",
"Rouen",
"Paris",
"Amiens",
"Lille",
"Strasbourg",
"Chateau du Haut Koenigsbourg",
"Colmar",
"Eguisheim",
"Besancon",
"Dijon",
"Annecy",
"Grenoble",
"Lyon",
"Gorges du Verdon",
"Bormes les Mimosas",
"Cassis",
"Marseille",
"Aix en Provence",
"Avignon",
"Uzes",
"Nimes",
"Aigues Mortes",
"Saintes Maries de la mer",
"Collioure",
"Carcassonne",
"Ariege",
"Toulouse",
"Montauban",
"Biarritz",
"Bayonne",
"La Rochelle"]

In [3]:
listlat=[]
listlon=[]
for i in range(len(listofcities)):
    payload={"q":listofcities[i],"country":"FR", "format":"json"}
    r= requests.get(url="https://nominatim.openstreetmap.org/", params=payload)
    print(r)
    print("i is {}.r is {}. latitude is {} and longitude is {}".format(i,r,r.json()[0]["lat"],r.json()[0]["lon"]))
    listlat.append(r.json()[0]["lat"])
    listlon.append(r.json()[0]["lon"])

<Response [200]>
i is 0.r is <Response [200]>. latitude is 48.6359541 and longitude is -1.511459954959514
<Response [200]>
i is 1.r is <Response [200]>. latitude is 48.649518 and longitude is -2.0260409
<Response [200]>
i is 2.r is <Response [200]>. latitude is 49.2764624 and longitude is -0.7024738
<Response [200]>
i is 3.r is <Response [200]>. latitude is 49.4938975 and longitude is 0.1079732
<Response [200]>
i is 4.r is <Response [200]>. latitude is 49.4404591 and longitude is 1.0939658
<Response [200]>
i is 5.r is <Response [200]>. latitude is 48.8588897 and longitude is 2.3200410217200766
<Response [200]>
i is 6.r is <Response [200]>. latitude is 49.8941708 and longitude is 2.2956951
<Response [200]>
i is 7.r is <Response [200]>. latitude is 50.6365654 and longitude is 3.0635282
<Response [200]>
i is 8.r is <Response [200]>. latitude is 48.584614 and longitude is 7.7507127
<Response [200]>
i is 9.r is <Response [200]>. latitude is 48.249489800000006 and longitude is 7.344296202531

In [4]:
len(listofcities)

35

In [5]:
payload={"q":listofcities[17],"country":"FR", "format":"json"}
r= requests.get(url="https://nominatim.openstreetmap.org/", params=payload)
r

<Response [200]>

In [6]:
dfcities = pd.DataFrame(
    {'Destination': listofcities,
     'Lat': listlat,
     'Lon': listlon
    })
dfcities = dfcities.astype({'Lat':'float'})
dfcities = dfcities.astype({'Lon':'float'})

In [7]:
dfcities.head()

,Destination,Lat,Lon
0,Mont Saint Michel,48.635954,-1.511460
1,St Malo,48.649518,-2.026041
2,Bayeux,49.276462,-0.702474
3,Le Havre,49.493898,0.107973
4,Rouen,49.440459,1.093966


In [8]:
len(dfcities)

35

In [39]:
dfkeys=pd.read_csv("keys.csv")

In [40]:
day=[]
averagefeltemperature=[]
percentagerain=[]
hoursofsun=[]
weathertype=[]
weathertypescore=[]
place=[]

for j in range(len(dfcities)):
    payload2={"units":"metric","appid":dfkeys["appid"][0],"exclude":"minutely,hourly","lat":round(dfcities["Lat"][j],2),"lon":round(dfcities["Lon"][j],2)}
    r2=requests.get(url="https://api.openweathermap.org/data/2.5/onecall", params=payload2)
    for i in range(7):
        day.append(i+1)
        place.append(dfcities["Destination"][j])
        averagefeltemperature.append(round(r2.json()["daily"][i]["feels_like"]["day"]*0.5+r2.json()["daily"][i]["feels_like"]["eve"]*0.25+r2.json()["daily"][i]["feels_like"]["morn"]*0.25,2))
        percentagerain.append(r2.json()["daily"][i]["pop"])
        hoursofsun.append(round((r2.json()["daily"][i]["sunset"]-r2.json()["daily"][i]["sunrise"])/3600,2))
        weathertype.append(r2.json()["daily"][i]["weather"][0]["description"])


In [36]:
dfweather= pd.DataFrame(
    {"Destination":place,
        'AverageFeltTemperature': averagefeltemperature,
    "Day": day,
     'HoursOfSun': hoursofsun,
     'Percentageofchanceofrain': percentagerain,
     "WeatherType":weathertype
    })


In [37]:
dfweather.head(10)

,Destination,AverageFeltTemperature,Day,HoursOfSun,Percentageofchanceofrain,WeatherType
0,Mont Saint Michel,11.88,1,14.61,0.51,light rain
1,Mont Saint Michel,13.26,2,14.66,0.08,few clouds
2,Mont Saint Michel,12.71,3,14.71,0.02,few clouds
3,Mont Saint Michel,13.43,4,14.76,0.00,overcast clouds
4,Mont Saint Michel,14.06,5,14.81,0.33,overcast clouds
5,Mont Saint Michel,13.62,6,14.86,0.41,light rain
6,Mont Saint Michel,15.84,7,14.91,0.00,overcast clouds
7,St Malo,11.88,1,14.61,0.91,light rain
8,St Malo,12.35,2,14.66,0.11,few clouds
9,St Malo,12.43,3,14.71,0.00,few clouds


In [12]:
snow=-8
dictweather={"thunderstorm with light rain":-3,"thunderstorm with rain":-5,"thunderstorm with heavy rain":-9,"light thunderstorm":1,"thunderstorm":-1,"heavy thunderstorm":-10,"ragged thunderstorm":-10,"thunderstorm with light drizzle":-2,"thunderstorm with drizzle":-3,"thunderstorm with heavy drizzle":-4,"light intensity drizzle":6,"drizzle":5,"heavy intensity drizzle":3,"light intensity drizzle rain":5,"drizzle rain":4,"heavy intensity drizzle rain":0,"shower rain and drizzle":snow,"heavy shower rain and drizzle":snow,"shower drizzle":1,"light rain":5,"moderate rain":3,"heavy intensity rain":1,"very heavy rain":0,"extreme rain":-4,"freezing rain":-4,"light intensity shower rain":4,"shower rain":3,"heavy intensity shower rain":0,"ragged shower rain":-2,"light snow":snow,"Snow":snow,"Heavy snow":-10,"Sleet":snow/2,"Light shower sleet":snow/2,"Shower sleet":snow/2,"Light rain and snow":snow/2,"Rain and snow":snow/2-3,"Light shower snow":snow/2,"Shower snow":snow,"Heavy shower snow":snow,"mist":5,"Smoke":-2,"Haze":3,"sand/ dust whirls":-10,"fog":2,"sand":-3,"dust":0,"volcanic ash":-50,"squalls":3,"tornado":-50,"clear sky":10,"few clouds":9,"scattered clouds":8,"broken clouds":7,"overcast clouds":6}
dfweather["WeatherTypeScore"]=dfweather["WeatherType"].map(dictweather)

In [14]:
rain=50
sun=1
tempinput=1
temp=tempinput*(snow-8)/-16
weatherimportance=1

dfweather["WeatherScore"]=dfweather["AverageFeltTemperature"]*temp+dfweather["Percentageofchanceofrain"]*rain+dfweather["HoursOfSun"]*sun+dfweather["WeatherTypeScore"]*weatherimportance


In [15]:
dfweatherweek=dfweather[["Destination","AverageFeltTemperature","HoursOfSun","Percentageofchanceofrain","WeatherTypeScore","WeatherScore"]].groupby("Destination").mean().round(2) 
dfweatherweek.head()

,AverageFeltTemperature,HoursOfSun,Percentageofchanceofrain,WeatherTypeScore,WeatherScore
Destination,,,,,
Aigues Mortes,17.19,14.18,0.20,6.71,47.88
Aix en Provence,17.82,14.18,0.11,7.00,44.50
Amiens,11.77,14.73,0.07,7.29,37.29
Annecy,13.97,14.37,0.51,6.00,59.84
Ariege,9.53,14.14,0.76,4.71,66.16


In [19]:
dfcitieswithweather= pd.merge(dfcities,dfweatherweek,on="Destination")
dfcitieswithweather["DestinationID"]=dfcitieswithweather.index
dfcitieswithweather.head()

,Destination,Lat,Lon,AverageFeltTemperature,HoursOfSun,Percentageofchanceofrain,WeatherTypeScore,WeatherScore,DestinationID
0,Mont Saint Michel,48.635954,-1.511460,12.46,14.61,0.10,6.14,38.43,0
1,St Malo,48.649518,-2.026041,11.72,14.61,0.02,6.57,34.04,1
2,Bayeux,49.276462,-0.702474,10.94,14.67,0.04,7.00,34.46,2
3,Le Havre,49.493898,0.107973,10.96,14.69,0.03,6.86,34.22,3
4,Rouen,49.440459,1.093966,12.07,14.68,0.12,6.71,39.32,4


In [20]:
dfcitieswithweather.to_csv("Citieswithweather.csv")

In [21]:

fig = px.scatter_mapbox(dfcitieswithweather, lat="Lat", lon="Lon", color="WeatherScore", size="WeatherScore", hover_name="Destination",hover_data={"WeatherScore":True,"AverageFeltTemperature":True,"HoursOfSun":True,"Percentageofchanceofrain":True,"WeatherTypeScore":True,"Lat":False,"Lon":False},zoom=4, mapbox_style="carto-positron", title="Map of 35 best destinations, ranked by best weather")
fig.show()

In [33]:
fig = px.scatter_mapbox(dfcitieswithweather.sort_values("WeatherScore").tail(5), lat="Lat", lon="Lon", color="WeatherScore", size="WeatherScore", hover_name="Destination",hover_data={"WeatherScore":True,"AverageFeltTemperature":True,"HoursOfSun":True,"Percentageofchanceofrain":True,"WeatherTypeScore":True,"Lat":False,"Lon":False},zoom=4, mapbox_style="carto-positron", title="Map of 5 best destinations, ranked by best weather")
fig.show()